In [ ]:
# Author: Arthur BARREAU
# Date: 2025-05-12
# R version: 4.3.3
# Description: This script generates monthly mean sea ice concentration maps (left column)
#              and associated anomalies (right column), using satellite data from the
#              National Snow and Ice Data Center (NSIDC). The pink line represents the
#              median sea ice extent during the reference period 1981–2010.
# Source: https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/

# ==============================================================================
# 0. PACKAGE & LIBRARY INSTALLATION ----
# ==============================================================================

options(timeout=3600)
system("conda install -y r-sf=1.0_20 r-terra=1.8_42 r-fields r-magick")
system("cp /opt/conda/pkgs/proj-9.6.0-h0054346_1/share/proj/proj.db /opt/conda/envs/rlang-kernel/share/proj/proj.db")

library(terra)
library(dplyr)
library(stringr)
library(RColorBrewer)
library(sf)
library(jsonlite) 
library(fields)
library(magick)

# ==============================================================================
# 1. VARIABLES ----
# ==============================================================================

json_data <- fromJSON("galaxy_inputs/galaxy_inputs.json")

start_month      <- json_data$month
start_year       <- json_data$year
chose            <- json_data$ConcentrationXandXorXAnomalies
asd_show         <- json_data$ASD
subarea          <- json_data$subarea
month_list       <- trimws(unlist(strsplit(start_month, ",")))
year_list        <- trimws(unlist(strsplit(start_year, ",")))
chose_list       <- trimws(unlist(strsplit(chose, "_")))
area_list        <- trimws(unlist(strsplit(subarea, ",")))
m                <- json_data$multiplier
text_title       <- json_data$title
legend_chose     <- json_data$legendXchose
bbox             <- json_data$bbox
year_ano         <- json_data$yearXanomaly
png              <- json_data$png
# ==============================================================================
# 2. VARIABLES ----
# ==============================================================================

months_days <- list(
  Jan = 31, Feb = 28, Mar = 31, Apr = 30, May = 31, Jun = 30, 
  Jul = 31, Aug = 31, Sep = 30, Oct = 31, Nov = 30, Dec = 31
)

# ==============================================================================
# 3.1 FUNCTIONS ----
# ==============================================================================

# Create directories for storing TIFF and Shapefile data
create_directories <- function(year, month) {
  tif_path <- sprintf("%s_%s_tif_folder", year,month)
  zip_path <- sprintf("%s_%s_zip_folder", year,month)
  shp_path <- sprintf("%s_%s_shapefile_folder", year,month)
  tif_mean_path <- sprintf("%s_%s_folder_tif_1981_2010", year,month)
  dir.create(tif_path, showWarnings = FALSE, recursive = TRUE)
  dir.create(zip_path, showWarnings = FALSE, recursive = TRUE)
  dir.create(shp_path, showWarnings = FALSE, recursive = TRUE)
  dir.create(tif_mean_path, showWarnings = FALSE, recursive = TRUE)
  return(list(tif_path = tif_path, zip_path = zip_path, shp_path = shp_path, tif_mean_path=tif_mean_path))
}

# Download daily TIFF files
download_tiff_files <- function(path, year, start_month, month_index) {
  month_index <- match(start_month, names(months_days))  
  month_folder <- sprintf("%02d_%s", month_index, start_month) 
  
  for (day in 1:months_days[[start_month]]) {
    date_str <- sprintf("%s%02d%02d", year, month_index, day)  
    url <- paste0("https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/", year, "/", month_folder, "/S_", date_str, "_concentration_v3.0.tif")
    destfile <- file.path(path, sprintf("S_%s_concentration.tif", date_str))
    tryCatch({
      download.file(url, destfile)
    },error=function(e){})
    
  }
}


# Compute the monthly mean raster
compute_monthly_mean <- function(path, start_year, month_index) {
  tif_files <- list.files(path, pattern = "\\.tif$", full.names = TRUE)
  rasters <- rast(tif_files)
  rasters[rasters == 2550] <- NA  # Remplacer les valeurs 2550 par NA
  mean_raster <- mean(rasters, na.rm = TRUE)  
  return(mean_raster)
}


# Download and process shapefiles
download_shapefiles <- function(zip_path, shp_path, month_index) {
  month_index <- sprintf("%02d", month_index) 
  url <- paste0("https://noaadata.apps.nsidc.org/NOAA/G02135/south/monthly/shapefiles/shp_median/median_extent_S_",
                month_index, "_1981-2010_polyline_v3.0.zip")
  destfile <- file.path(zip_path, sprintf("median_extent_S_%s.zip", month_index))
  tryCatch({
    download.file(url, destfile)
    unzip(destfile, exdir = shp_path) 
  },error=function(e){})
}

# Merge shapefiles and compute mean polygon
process_shapefiles <- function(shp_path) {
  shp_files <- list.files(shp_path, pattern = "\\.shp$", full.names = TRUE)
  polygons <- lapply(shp_files, st_read)
  return(polygons)
}

download_tiff_1981_2010 <- function(path, start_year, end_year, month_index) {
  month_index <- sprintf("%02d", month_index) 
  base_url <- "https://noaadata.apps.nsidc.org/NOAA/G02135/south/monthly/geotiff/"
  
  months <- c("01_Jan", "02_Feb", "03_Mar", "04_Apr", "05_May", "06_Jun",
              "07_Jul", "08_Aug", "09_Sep", "10_Oct", "11_Nov", "12_Dec")
  month_folder <- months[as.integer(month_index)]
  
  for (year in start_year:end_year) {
    date_str <- sprintf("S_%s%s_concentration_v3.0.tif", year, month_index)
    url <- paste0(base_url, month_folder, "/", date_str)
    destfile <- file.path(path, date_str)
    tryCatch({
      download.file(url, destfile, mode = "wb")
    },error=function(e){})
  }
}

compute_mean_raster_1981_2010 <- function(path,month_index) { 
  month_index <- sprintf("%02d", month_index) 
  files <- list.files(path, pattern = "\\.tif$", full.names = TRUE)
  rasters <- rast(files)
  mean_raster_1981_2010 <- mean(rasters, na.rm = TRUE)  
  return(mean_raster_1981_2010)
}

# Create color palettes and breaks for visualization
create_colors_and_breaks <- function() {
  breaks <- c(0,1, seq(150, 1000, by = 50),1050, 2531, 2541)
  colors_glacier <- rev(c("#F7FCFF", "#E4F4FE", "#D0ECFE", "#BCE4FE", "#A8DCFD",
                          "#94D5FD", "#7DCCFD", "#6AC4FC", "#57BCFC", "#45B4FC",
                          "#31ABFC", "#23A3FC", "#1A9BFC", "#1994F9", "#178CF2",
                          "#1684EB", "#137AE3", "#093C70","#093C70" ))
  colors <- c(
    rgb(9, 60, 112, maxColorValue = 255),
    colors_glacier,  # Dégradé bleu glacier (1 à 1000)
    rgb(0, 0, 0, maxColorValue = 255),      # Ligne côtière noire (2530)
    rgb(119, 119, 119, maxColorValue = 255) # Terre grise (2540)
  )
  return(list(colors = colors, breaks = breaks))
}

# Visualization and saving results
save_plot <- function(raster, colors, breaks, polygons, year, month,subareas_selected,area_list,title,ASD) {
  plot(raster, col = colors, breaks = breaks, legend = FALSE, axes = FALSE, box = FALSE,mar = c(0, 0, 0, 0))
  plot(st_geometry(polygons[[1]]), add = TRUE, pch = 21, col = 'red', cex = 1.5*m)
  if(ASD){
    plot(st_geometry(subareas_selected), add = TRUE, lwd = 1.5*m, border = 'red',mar=c(0,0,0,0))
    zones <- st_read("asd/asd-shapefile-EPSG6932.shp")  
    zones_to_plot <- if (is.na(letters_only[1])) {area_list} else {default_areas}
    zones <- zones %>% filter(GAR_Short_ %in% zones_to_plot) 
    centroids <- st_centroid(st_geometry(zones))
    text(st_coordinates(centroids), labels = zones$GAR_Long_L, col = "red", cex = 0.8*m) 
  }
  
  
  #add_labels(mode = 'auto', layer = 'ASDs', fontsize =1, col = 'red')
  if(text_title != "No Title"){
      if(text_title == "Title in the Middle"){
        text(x = 350000, y = 150000, labels = sprintf("%s_%s", year, month), cex = 3*m, font = 1, col="white")
          
      }else{
          text(x = 350000, y = 4000000, labels = sprintf("%s_%s", year, month), cex = 3*m, font = 1,col="white") 
      }
  }  
}

# ==============================================================================
# 3.2 ANOMALY ----
# ==============================================================================

filter_raster_values <- function(raster) {
  raster[] <- ifelse(raster[] >= 1 & raster[] <= 150, 0, raster[])
  return(raster)
}

compute_anomaly <- function(mean_raster, mean_1981_2010) {
  mean_raster <- filter_raster_values(mean_raster)
  mean_1981_2010 <- filter_raster_values(mean_1981_2010)
  
  anomaly <- mean_raster - mean_1981_2010
  
  minmax_value <- minmax(anomaly)
  min_anomaly <- minmax_value[1]
  max_anomaly <- minmax_value[2]
  
  breaks_neg <- seq(-500, -1, length.out = 11)  
  breaks_pos <- seq(1, 500, length.out = 11) 
  breaks <- unique(c(min_anomaly, breaks_neg, breaks_pos, max_anomaly))
  col_fun <- c("#053061", "#114781", "#1D5FA2", "#2B73B3", "#3A87BD", "#4F9BC7", "#71B0D3", 
               "#93C6DE", "#B1D5E7", "#CCE2EF", "#DEEBF2", "white", "#F8F1ED", "#FBE5D8", "#FCD7C2", 
               "#F8BFA4", "#F4A683", "#E8896C", "#DB6B55", "#CC4C44", "#BD2D35", "#A81529", 
               "#870A24")
  
  return(list(anomaly = anomaly, breaks = breaks, col_fun = col_fun))
}

save_anomaly_plot <- function(anomaly_data,mean_raster, year, month,subareas_selected,area_list,ASD) {
    plot(anomaly_data$anomaly, col = anomaly_data$col_fun, breaks = anomaly_data$breaks, legend = FALSE, axes = FALSE, box = FALSE, mar = c(0, 0, 0, 0))
    plot(mean_raster, col = c(rgb(0, 0, 0, maxColorValue = 255),rgb(119, 119, 119, maxColorValue = 255)), breaks = c(2529,2531,2540),add=TRUE, legend = FALSE, axes = FALSE, box = FALSE)
    if(ASD){
    plot(st_geometry(subareas_selected), add = TRUE, lwd = 1.5*m, border = 'red',mar=c(0,0,0,0))
    zones <- st_read("asd/asd-shapefile-EPSG6932.shp")    
    zones_to_plot <- if (is.na(letters_only[1])) {area_list} else {default_areas}
    zones <- zones %>% filter(GAR_Short_ %in% zones_to_plot) 
    centroids <- st_centroid(st_geometry(zones))
    text(st_coordinates(centroids), labels = zones$GAR_Long_L, col = "red", cex = 0.8*m)  # Ajouter les labels
    }
  
    if(text_title != "No Title"){
      if(text_title == "Title in the Middle"){
          text(x = 350000, y = 150000, labels = sprintf("%s_%s", year, month), cex = 3*m, font = 1, col="white")
      }else{
          
          text(x = 350000, y = 4000000, labels = sprintf("%s_%s", year, month), cex = 3*m, font = 1) 
      }  
  }  
}

mask_subarea <- function(area_list, raster, ASD) {
    if (is.na(letters_only[1])) {
        output_dir <- "asd"
        dir.create(output_dir, showWarnings = FALSE)
        base_url <- "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG6932"
        extensions <- c(".shp", ".shx", ".dbf", ".prj", ".cst")
        
        urls_asd <- paste0(base_url, extensions)
        destfiles <- file.path(output_dir, paste0("asd-shapefile-EPSG6932", extensions))
        
        # Download each file and save to the output directory
        for (i in seq_along(urls_asd)) {
            download.file(urls_asd[i], destfiles[i], mode = "wb")
        }
        
        ASD <- st_read("asd/asd-shapefile-EPSG6932.shp", quiet = TRUE)
        subareas_selected <- ASD %>% filter(GAR_Short_ %in% area_list)
        
        if(bbox){
          subareas_bbox <- vect(st_as_sfc(st_bbox(subareas_selected)))
          mask_mean_raster <- mask(crop(raster, subareas_bbox), subareas_bbox)
        } else{
            mask_mean_raster <- mask(crop(raster, vect(subareas_selected)), vect(subareas_selected))
        }

        # Preserve land and coast values
        land_coast <- raster
        land_coast[!(raster[] %in% c(2530, 2540))] <- NA  # Keep only pixels 2530 and 2540
        land_coast_mask <- mask(crop(land_coast, ext(vect(subareas_selected))), ext(vect(subareas_selected)))

        # Combine both: masked concentration + preserved land/coast layer
        mask_mean_raster <- cover(mask_mean_raster, land_coast_mask)

        return(list(mean_raster = mask_mean_raster, subareas_selected = subareas_selected))

    } else {
        bounds <- list(
            N = as.numeric(numbers_only[letters_only == "N"]),
            S = as.numeric(numbers_only[letters_only == "S"]),
            W = as.numeric(numbers_only[letters_only == "W"]),
            E = as.numeric(numbers_only[letters_only == "E"])
        )
        zone_extent <- ext(bounds$W, bounds$E, bounds$S, bounds$N)
        projected_zone <- project(rast(ext = zone_extent, crs = "EPSG:4326"), "EPSG:6932")
        area_vector <- NULL
        extent_obj <- ext(projected_zone)
        mask_mean_raster <- mask(crop(raster, extent_obj), extent_obj)

        if (ASD) {
            output_dir <- "asd"
            dir.create(output_dir, showWarnings = FALSE)
            base_url <- "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG6932"
            extensions <- c(".shp", ".shx", ".dbf", ".prj", ".cst")

            urls_asd <- paste0(base_url, extensions)
            destfiles <- file.path(output_dir, paste0("asd-shapefile-EPSG6932", extensions))

            # Download each file and save to the output directory
            for (i in seq_along(urls_asd)) {
                download.file(urls_asd[i], destfiles[i], mode = "wb")
            }

            ASD <- st_read("asd/asd-shapefile-EPSG6932.shp", quiet = TRUE)
            subareas_selected <- ASD
            return(list(mean_raster = mask_mean_raster, subareas_selected = subareas_selected))
        }

        return(mask_mean_raster)
    }
}


# ==============================================================================
# 4. MAIN ----
# ==============================================================================

count            <- 0

if(is.null(year_ano)){
    start_year = 1981
    end_year   = 2010
}else{
    year_ano <- as.numeric(trimws(unlist(strsplit(year_ano, "_"))))
    start_year = year_ano[1]
    end_year   = year_ano[2]
}

default_areas <- c('883', '882', '481', '484', '482', '483', '5843a', '5843b','5852', '485', '486', '5841', '5842', '881', '5844a', '587','586', '5851', '5844b')

letters_only <- str_extract(area_list, "[A-Za-z]+")
numbers_only <- str_extract(area_list, "-?\\d+")

for(i in seq_along(month_list)) {
  paths <- create_directories(year_list[i], month_list[i])
  month_index <- match(month_list[i], names(months_days))
  
  download_tiff_files(paths$tif_path, year_list[i], month_list[i], month_index)
  mean_raster <- compute_monthly_mean(paths$tif_path, year_list[i], month_index)
  

  mask_return <- mask_subarea(area_list, mean_raster,asd_show)
  if (is.list(mask_return)){
    subareas_selected <- mask_return$subareas_selected
    mean_raster_mask <- mask_return$mean_raster
  }else{
    subareas_selected <- FALSE
    mean_raster_mask <- mask_return
  }

  
  download_shapefiles(paths$zip_path, paths$shp_path, month_index)
  polygons <- process_shapefiles(paths$shp_path)
  color_breaks <- create_colors_and_breaks()
  
  if(count == 0){
    raster_extent <- ext(mean_raster_mask)
    w <- abs(raster_extent[2] - raster_extent[1]) / 10000
    h <- abs(raster_extent[4] - raster_extent[3]) / 10000
    if (png) {
      png("outputs/collection/Fig3.png", width = w * length(chose_list) * m, height = h * length(month_list) * m)   
    } else {
      pdf("outputs/collection/Fig3.pdf", width = w * length(chose_list) * m, height = h * length(month_list) * m)
        m <- m*100
    }
    par(mfrow = c(length(month_list), length(chose_list)))
    count <- 1
  }
  
  for(txt in chose_list){
    if(txt == "Concentration"){
      save_plot(mean_raster_mask, color_breaks$colors, color_breaks$breaks, polygons, year_list[i], month_list[i], subareas_selected,area_list, text_title,asd_show)
    }
    if(txt == "Anomalies"){
        download_tiff_1981_2010(paths$tif_mean_path, start_year, end_year, month_index)
        mean_1981_2010 <- compute_mean_raster_1981_2010(paths$tif_mean_path, month_index)
        mean_1981_2010_mask <- mask_subarea(area_list, mean_1981_2010,asd_show)
        if (is.list(mean_1981_2010_mask)) {
          mean_1981_2010_raster <- mean_1981_2010_mask$mean_raster
        } else {
          mean_1981_2010_raster <- mean_1981_2010_mask
        }
        anomaly_data <- compute_anomaly(mean_raster_mask, mean_1981_2010_raster)
        save_anomaly_plot(anomaly_data, mean_raster_mask, year_list[i], month_list[i], subareas_selected,area_list,asd_show)
    }
  }
}
dev.off()

In [ ]:
# ==============================================================================
# 5. LEGEND ----
# ==============================================================================


path_conc <- "outputs/Legend_Fig3_Concentration.png"
path_ano <- "outputs/Legend_Fig3_Anomalies.png"

generate_legend <- function(zlim, breaks, colors, horizontal = TRUE, png_filename = "legend.png") {
  # Define the limits of the legend
  labels_legend <- format(zlim, nsmall = 2)  # Use zlim here for labels
  n <- 5  # size factor

  # Set image dimensions based on orientation
  if (horizontal) {
    png(filename = png_filename, width = 400 * n, height = 75 * n, bg = "transparent")
    par(mar = c(2, 5, 2, 2))  # margins for horizontal legend
  } else {
    png(filename = png_filename, width = 75 * n, height = 400 * n, bg = "transparent")
    par(mar = c(5, 2, 2, 5))  # margins for vertical legend
  }

  # Plot the legend only
  fields::image.plot(
    zlim = zlim,
    breaks = breaks,
    col = colors,
    legend.only = TRUE,
    horizontal = horizontal,
    legend.width =  1.4 * n,
    legend.shrink = 0.9,
    legend.mar = 8,
    axis.args = list(
      at = zlim,  # Use zlim for tick positions
      labels = labels_legend,  # Labels based on zlim
      cex.axis = 2,
      col.axis = "black"
    )
  )

  dev.off()
}

if(legend_chose != "No legend"){
    if(legend_chose == "Horizontal"){
        horizontal = TRUE        
    }else{
        horizontal = FALSE        
    }
    for(txt in chose_list){
        if(txt == "Concentration"){
            zlim <- seq(from = 0, to = 100, by = 10)
            breaks <- seq(0,100,by=5)
            colors <- color_breaks$colors
            colors <- colors[1:(length(colors) - 2)]
            generate_legend(zlim,breaks, colors, horizontal, png_filename = path_conc)    
        }
        if(txt == "Anomalies"){
            zlim <- seq(from = -50, to = 50, by = 10)
            breaks <- seq(-55,55,by=5)
            colors <- setdiff(anomaly_data$col_fun,"white")
            generate_legend(zlim,breaks, colors, horizontal, png_filename = path_ano)
        }
    }
}

if (chose == "Concentration_Anomalies" ){     
    
    img_conc <- image_read(path_conc)
    file.remove(path_conc)
                
    img_ano <- image_read(path_ano)
    file.remove(path_ano)
    
    # Combiner horizontalement avec un petit espace entre les deux
    combined <- image_append(c(img_conc, img_ano), stack = FALSE)
    # Sauvegarder
    image_write(combined, "outputs/Legend_Fig3_Combined.png")

}